In [5]:
!pip install matplotlib pandas requests python-dotenv
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt



[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import os
from dotenv import load_dotenv

# Загружаем переменные окружения из .env
load_dotenv()

API_URL = os.getenv("API_URL")
DATE_BEGIN = os.getenv("DATE_BEGIN")
DATE_END = os.getenv("DATE_END")

print("API_URL:", API_URL)
print("DATE_BEGIN:", DATE_BEGIN)
print("DATE_END:", DATE_END)

API_URL: https://data-charts-api.hexlet.app
DATE_BEGIN: 2023-03-01
DATE_END: 2023-09-01


In [7]:
import os, json, time
import pandas as pd
import requests
from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter

# ---------- вспомогательные функции ----------
def month_ranges(begin: str, end: str):
    b = pd.to_datetime(begin).normalize()
    e = pd.to_datetime(end).normalize()
    starts = pd.date_range(b, e, freq="MS")
    if len(starts) == 0 or starts[0] != b:
        starts = starts.insert(0, b)
    ends = list(starts[1:] - pd.Timedelta(days=1)) + [e]
    pairs = []
    for s, t in zip(starts, ends):
        if s > e:
            break
        t = min(t, e)
        if s <= t:
            pairs.append((s.strftime("%Y-%m-%d"), t.strftime("%Y-%m-%d")))
    return pairs

def make_session(total=4, backoff=0.5, timeout=(5, 120)):
    sess = requests.Session()
    retry = Retry(
        total=total,
        read=total,
        connect=total,
        backoff_factor=backoff,
        status_forcelist=(429, 500, 502, 503, 504),
        allowed_methods=frozenset(["GET"]),
        raise_on_status=False,
    )
    adapter = HTTPAdapter(max_retries=retry, pool_connections=10, pool_maxsize=10)
    sess.mount("https://", adapter)
    sess.mount("http://", adapter)
    sess.request_timeout = timeout
    return sess

def fetch_table_chunked(api_url_base: str, route: str, begin: str, end: str, session: requests.Session):
    frames = []
    for b, e in month_ranges(begin, end):
        r = session.get(f"{api_url_base}/{route}", params={"begin": b, "end": e}, timeout=session.request_timeout)
        r.raise_for_status()
        data = r.json()
        if data:
            frames.append(pd.DataFrame(data))
        time.sleep(0.1)  # бережно к API
    return pd.concat(frames, ignore_index=True) if frames else pd.DataFrame()

# ---------- основная загрузка с фоллбэком ----------
print("Пробую загрузить из API с ретраями…")
sess = make_session()

try:
    # VISITS
    visits_api = fetch_table_chunked(API_URL, "visits", DATE_BEGIN, DATE_END, sess) \
        .rename(columns={"visit_id": "uuid", "datetime": "date"})
    visits_api["date"] = pd.to_datetime(visits_api["date"], errors="coerce")
    for c in ["uuid", "platform", "user_agent"]:
        if c in visits_api.columns:
            visits_api[c] = visits_api[c].astype(str).str.strip().str.lower()

    # REGS
    regs_api = fetch_table_chunked(API_URL, "registrations", DATE_BEGIN, DATE_END, sess) \
        .rename(columns={"datetime": "date"})
    regs_api["date"] = pd.to_datetime(regs_api["date"], errors="coerce")
    for c in ["user_id", "email", "platform", "registration_type"]:
        if c in regs_api.columns:
            regs_api[c] = regs_api[c].astype(str).str.strip().str.lower()

    print("OK из API:", visits_api.shape, regs_api.shape)

except Exception as e:
    print("⚠️ Не удалось получить из API:", repr(e))
    print("Пробую взять локальные файлы…")

    # 1) processed CSV (твои чистые выгрузки)
    v_csv = "./data/processed/visits_api_clean.csv"
    r_csv = "./data/processed/registrations_api_clean.csv"
    if os.path.exists(v_csv) and os.path.exists(r_csv):
        visits_api = pd.read_csv(v_csv)
        regs_api = pd.read_csv(r_csv)
        visits_api["date"] = pd.to_datetime(visits_api["date"], errors="coerce")
        regs_api["date"] = pd.to_datetime(regs_api["date"], errors="coerce")
        print("OK из processed CSV:", visits_api.shape, regs_api.shape)
    else:
        # 2) raw JSON (есть в data/raw)
        v_json = f"./data/raw/visits_{DATE_BEGIN}_{DATE_END}.json"
        r_json = f"./data/raw/registrations_{DATE_BEGIN}_{DATE_END}.json"
        # если имена другие — подхватим любые подходящие
        if not os.path.exists(v_json):
            cand = [p for p in os.listdir("./data/raw") if p.startswith("visits_") and p.endswith(".json")]
            if cand: v_json = os.path.join("./data/raw", cand[0])
        if not os.path.exists(r_json):
            cand = [p for p in os.listdir("./data/raw") if p.startswith("registrations_") and p.endswith(".json")]
            if cand: r_json = os.path.join("./data/raw", cand[0])

        if os.path.exists(v_json) and os.path.exists(r_json):
            visits_api = pd.DataFrame(json.load(open(v_json, "r", encoding="utf-8"))).rename(
                columns={"visit_id": "uuid", "datetime": "date"}
            )
            regs_api = pd.DataFrame(json.load(open(r_json, "r", encoding="utf-8"))).rename(
                columns={"datetime": "date"}
            )
            visits_api["date"] = pd.to_datetime(visits_api["date"], errors="coerce")
            regs_api["date"] = pd.to_datetime(regs_api["date"], errors="coerce")
            for c in ["uuid", "platform", "user_agent"]:
                if c in visits_api.columns:
                    visits_api[c] = visits_api[c].astype(str).str.strip().str.lower()
            for c in ["user_id", "email", "platform", "registration_type"]:
                if c in regs_api.columns:
                    regs_api[c] = regs_api[c].astype(str).str.strip().str.lower()
            print("OK из raw JSON:", visits_api.shape, regs_api.shape)
        else:
            raise RuntimeError("Нет ни API, ни локальных файлов для загрузки данных.")



Пробую загрузить из API с ретраями…
OK из API: (255028, 4) (21094, 5)


In [16]:
import numpy as np
# Преобразуем колонки к удобному виду и собираем данные из API
params = {"begin": DATE_BEGIN, "end": DATE_END}

# --- Визиты ---
resp_v = requests.get(f"{API_URL}/visits", params=params, timeout=60)
resp_v.raise_for_status()
visits_api = pd.DataFrame(resp_v.json()).rename(columns={"visit_id": "uuid", "datetime": "date"})
visits_api["date"] = pd.to_datetime(visits_api["date"], errors="coerce")
for c in ["uuid", "platform", "user_agent"]:
    visits_api[c] = visits_api[c].astype(str).str.strip().str.lower()

# --- Регистрации ---  <-- ЭТОГО БЛОКА НЕ ХВАТАЛО
resp_r = requests.get(f"{API_URL}/registrations", params=params, timeout=60)
resp_r.raise_for_status()
regs_api = pd.DataFrame(resp_r.json()).rename(columns={"datetime": "date"})
regs_api["date"] = pd.to_datetime(regs_api["date"], errors="coerce")
for c in ["user_id", "email", "platform", "registration_type"]:
    regs_api[c] = regs_api[c].astype(str).str.strip().str.lower()

# === 1) Готовим визиты и регистрации (работаем от visits_api / regs_api) ===
visits = visits_api.copy()
regs   = regs_api.copy()

# исключаем ботов (по user_agent) из расчёта визитов
visits["is_bot"] = visits["user_agent"].str.contains("bot", na=False)
visits_nobots = visits.loc[~visits["is_bot"]].copy()

# последний визит на пользователя
visits_nobots = visits_nobots.sort_values("date")
last_visits = (
    visits_nobots.groupby("uuid", as_index=False)
    .agg({"date": "max"})
)
visits_last = last_visits.merge(
    visits_nobots[["uuid", "date", "platform"]],
    on=["uuid", "date"],
    how="left"
)

# группируем по дням/платформам
visits_last["date_group"] = visits_last["date"].dt.date
visits_daily = (
    visits_last.groupby(["date_group", "platform"], as_index=False)
    .agg(visits=("uuid", "count"))
)

regs["date_group"] = regs["date"].dt.date
regs_daily = (
    regs.groupby(["date_group", "platform"], as_index=False)
    .agg(registrations=("user_id", "count"))
)

# объединяем и считаем конверсию
merged = (
    visits_daily.merge(regs_daily, on=["date_group", "platform"], how="outer")
    .fillna({"visits": 0, "registrations": 0})
)
merged["visits"] = merged["visits"].astype(int)
merged["registrations"] = merged["registrations"].astype(int)

# без округления!
conv = merged["registrations"] / merged["visits"].replace(0, pd.NA)
merged["conversion"] = (conv * 100).astype(float)
merged["conversion"] = merged["conversion"].fillna(0)


# сохраняем conversion.json (требование автотестов)
merged[["date_group", "platform", "visits", "registrations", "conversion"]].to_json("./conversion.json")
print("saved: ./conversion.json")

# === 2) Ежедневные итоги и объединение с рекламой ===
try:
    ads = pd.read_csv("./ads.csv")
    ads.columns = [c.strip().lower() for c in ads.columns]
    ads["date"] = pd.to_datetime(ads["date"], errors="coerce")
    ads["date_group"] = ads["date"].dt.date
    ads_grouped = ads.groupby(["date_group", "utm_campaign"], as_index=False).agg(cost=("cost", "sum"))
except FileNotFoundError:
    ads_grouped = pd.DataFrame(columns=["date_group", "utm_campaign", "cost"])

conv_daily = merged.groupby("date_group", as_index=False)[["visits", "registrations"]].sum()

final = conv_daily.merge(ads_grouped, on="date_group", how="left")
final["utm_campaign"] = final["utm_campaign"].fillna("none")
final["cost"] = final["cost"].fillna(0).astype(float)

# сохраняем ads.json (требование автотестов)
final[["date_group", "visits", "registrations", "cost", "utm_campaign"]].to_json("./ads.json")
print("saved: ./ads.json")

merged.head(), final.head()


saved: ./conversion.json
saved: ./ads.json


(   date_group platform  visits  registrations  conversion
 0  2023-03-01  android      75             61   81.333333
 1  2023-03-01      ios      22             18   81.818182
 2  2023-03-01      web     279              8    2.867384
 3  2023-03-02  android      67             59   88.059701
 4  2023-03-02      ios      31             24   77.419355,
    date_group  visits  registrations                utm_campaign   cost
 0  2023-03-01     376             87  advanced_algorithms_series  212.0
 1  2023-03-02     613            106  advanced_algorithms_series  252.0
 2  2023-03-03     683            107  advanced_algorithms_series  202.0
 3  2023-03-04     647            159  advanced_algorithms_series  223.0
 4  2023-03-05     707            115  advanced_algorithms_series  265.0)

In [ ]:
print("Visits shape:", visits.shape)
print("Registrations shape:", regs.shape)

print("Доля дубликатов в visits:", visits.duplicated().mean())
print("Доля дубликатов в registrations:", regs.duplicated().mean())

In [ ]:
print("Пропуски в visits:")
print(visits.isna().mean())

print("\nПропуски в registrations:")
print(regs.isna().mean())

In [ ]:
print("Visits by platform:")
print(visits["platform"].value_counts())

print("\nRegistrations by platform:")
print(regs["platform"].value_counts())

print("\nRegistration types:")
print(regs["registration_type"].value_counts())

In [ ]:
import pandas as pd
import requests

BASE = "https://data-charts-api.hexlet.app"
BEGIN = "2023-03-01"
END   = "2023-09-01"
params = {"begin": BEGIN, "end": END}
params

In [ ]:
print("visits_api:", visits_api.shape, 
      "| nulls:", visits_api.isna().mean().round(4).to_dict())
print("regs_api:", regs_api.shape, 
      "| nulls:", regs_api.isna().mean().round(4).to_dict())

print("\nVisits platforms:\n", visits_api["platform"].value_counts().head(10))
print("\nRegs platforms:\n", regs_api["platform"].value_counts().head(10))
print("\nRegs types:\n", regs_api["registration_type"].value_counts().head(10))


In [ ]:
# убираем визиты ботов
visits_clean = visits_api[~visits_api["user_agent"].str.contains("bot", case=False, na=False)].copy()

# для каждого uuid оставим только последний визит
visits_clean = visits_clean.sort_values("date").drop_duplicates("uuid", keep="last")

# агрегируем: количество визитов по дате и платформе
visits_grouped = visits_clean.groupby([visits_clean["date"].dt.date, "platform"]).size().reset_index(name="visits")

visits_grouped = visits_grouped.rename(columns={"date": "date_group"})
visits_grouped.head()

In [ ]:
# агрегируем регистрации: количество по дате и платформе
regs_grouped = regs_api.groupby([regs_api["date"].dt.date, "platform"]).size().reset_index(name="registrations")

regs_grouped = regs_grouped.rename(columns={"date": "date_group"})
regs_grouped.head()

In [ ]:
# объединение по дате и платформе
merged = pd.merge(visits_grouped, regs_grouped, on=["date_group", "platform"], how="outer")

# если где-то нет регистраций или визитов → ставим 0
merged = merged.fillna(0)

# конверсия = (registrations / visits * 100)
merged["conversion"] = (
    merged["registrations"] / merged["visits"].replace(0, pd.NA) * 100
).fillna(0)

# сортируем по дате
merged = merged.sort_values("date_group").reset_index(drop=True)

merged.head(10)


In [15]:
# Убираем ботов и берём последний визит на uuid
visits_clean = visits_api[~visits_api["user_agent"].str.contains("bot", case=False, na=False)].copy()
visits_clean = visits_clean.sort_values("date").drop_duplicates("uuid", keep="last")

# Агрегации визитов и регистраций по дате и платформе
vg = visits_clean.groupby([visits_clean["date"].dt.date, "platform"]).size().reset_index(name="visits")
vg = vg.rename(columns={"date": "date_group"})

rg = regs_api.groupby([regs_api["date"].dt.date, "platform"]).size().reset_index(name="registrations")
rg = rg.rename(columns={"date": "date_group"})

# Итоговый датафрейм с конверсией
merged = pd.merge(vg, rg, on=["date_group", "platform"], how="outer").fillna(0)
merged["conversion"] = (
    merged["registrations"] / merged["visits"].replace(0, pd.NA) * 100
).replace([float("inf"), -float("inf")], 0).fillna(0)

# Сводка по дням без платформ (для рекламы)
conv_daily = merged.groupby("date_group", as_index=False)[["visits","registrations"]].sum()

merged.head(), conv_daily.head()

(   date_group platform  visits  registrations  conversion
 0  2023-03-01  android      75             61   81.333333
 1  2023-03-01      ios      22             18   81.818182
 2  2023-03-01      web     279              8    2.867384
 3  2023-03-02  android      67             59   88.059701
 4  2023-03-02      ios      31             24   77.419355,
    date_group  visits  registrations
 0  2023-03-01     376             87
 1  2023-03-02     613            106
 2  2023-03-03     683            107
 3  2023-03-04     647            159
 4  2023-03-05     707            115)

In [ ]:
ads = pd.read_csv("./ads.csv")
ads.columns = [c.strip().lower() for c in ads.columns]
ads["date"] = pd.to_datetime(ads["date"], errors="coerce")
ads["date_group"] = ads["date"].dt.date

ads_grouped = ads.groupby(["date_group","utm_campaign"], as_index=False)["cost"].sum()

# Объединение с дневной статистикой
final = conv_daily.merge(ads_grouped, on="date_group", how="left")
final["utm_campaign"] = final["utm_campaign"].fillna("none")
final["cost"] = final["cost"].fillna(0)

final.head()

In [13]:
merged[["date_group","platform","visits","registrations","conversion"]].to_json(
    "./conversion.json", double_precision=10
)
print("saved: ./conversion.json")

saved: ./conversion.json


In [ ]:
# 1) читаем файл рекламы
# если вдруг у тебя разделитель ; (точка с запятой), смени на: pd.read_csv("../data/raw/ads.csv", sep=";")
ads = pd.read_csv("./ads.csv")   # файл лежит рядом с ноутбуком
ads.columns = [c.strip().lower() for c in ads.columns]
ads["date"] = pd.to_datetime(ads["date"], errors="coerce")
ads["date_group"] = ads["date"].dt.date

# 2) приведение типов/колонок
# ожидаемые колонки: date, utm_source, utm_medium, utm_campaign, cost
ads.columns = [c.strip().lower() for c in ads.columns]
ads["date"] = pd.to_datetime(ads["date"], errors="coerce")
ads["date_group"] = ads["date"].dt.date

# 3) оставим только нужные поля (для задачи нам нужны дата, кампания, затраты)
ads = ads[["date_group", "utm_campaign", "cost"]]

ads.head()

In [ ]:
ads_grouped = (
    ads.groupby(["date_group", "utm_campaign"], as_index=False)["cost"]
       .sum()
)
ads_grouped.head()


In [ ]:
# если у тебя уже есть merged из предыдущего шага
conv_daily = merged.groupby("date_group", as_index=False)[["visits", "registrations"]].sum()
conv_daily.head()

In [ ]:
final = conv_daily.merge(ads_grouped, on="date_group", how="left")

# если рекламы в этот день не было — ставим «none» и 0
final["utm_campaign"] = final["utm_campaign"].fillna("none")
final["cost"] = final["cost"].fillna(0)

# сортируем по дате
final = final.sort_values("date_group").reset_index(drop=True)
final.head(10)

In [ ]:
final[["date_group","visits","registrations","cost","utm_campaign"]].to_json(
    "./ads.json",
    double_precision=10
)

In [ ]:
from pathlib import Path
import matplotlib.pyplot as plt

CHARTS = Path("./charts")        # ВАЖНО: ровно ./charts
CHARTS.mkdir(parents=True, exist_ok=True)

plt.figure()
plt.plot([0, 1], [0, 1])         # любой простой график
plt.title("smoke chart")
plt.tight_layout()
plt.savefig(CHARTS / "_smoke.png", dpi=120)
plt.close()
print("chart saved to", CHARTS / "_smoke.png")

In [12]:
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

# где сохраняем (Важно: ровно ./charts)
CHARTS = Path("./charts")          # <-- строго ./charts
CHARTS.mkdir(parents=True, exist_ok=True)


# ВОССТАНОВЛЕНИЕ ДАННЫХ (если переменных нет в памяти)
# merged: по дням и платформам (date_group, platform, visits, registrations, conversion)
try:
    merged
except NameError:
    # пересоберём из visits_api / regs_api
    # 1) визиты без ботов, последний визит на uuid
    visits_clean = visits_api[~visits_api["user_agent"].str.contains("bot", case=False, na=False)].copy()
    visits_clean = visits_clean.sort_values("date").drop_duplicates("uuid", keep="last")
    vg = visits_clean.groupby([visits_clean["date"].dt.date, "platform"]).size().reset_index(name="visits")
    vg = vg.rename(columns={"date": "date_group"})

    # 2) регистрации
    rg = regs_api.groupby([regs_api["date"].dt.date, "platform"]).size().reset_index(name="registrations")
    rg = rg.rename(columns={"date": "date_group"})


    import numpy as np  # добавь выше, если не импортировано
    # 3) объединяем и считаем конверсию
    merged = pd.merge(vg, rg, on=["date_group", "platform"], how="outer").fillna(0)

    # без округления до 2 знаков (как требуют тесты)
    merged["conversion"] = (
        merged["registrations"] / merged["visits"].replace(0, np.nan) * 100
    )
    merged["conversion"] = merged["conversion"].fillna(0)



# сводка по дням (без платформ)
conv_daily = merged.groupby("date_group", as_index=False)[["visits","registrations"]].sum()

# final: conv_daily + реклама (date_group, visits, registrations, cost, utm_campaign)
try:
    final
except NameError:
    # читаем РОВНО из ./ads.csv (так ждут тесты)
    ads = pd.read_csv("./ads.csv")
    ads["date"] = pd.to_datetime(ads["date"], errors="coerce")
    ads["date_group"] = ads["date"].dt.date
    ads_grouped = ads.groupby(["date_group","utm_campaign"], as_index=False)["cost"].sum()
    final = conv_daily.merge(ads_grouped, on="date_group", how="left")
    final["utm_campaign"] = final["utm_campaign"].fillna("none")
    final["cost"] = final["cost"].fillna(0)

# вспомогательные функции для подписей
def annotate_bars(ax, fmt="{:,.0f}", rotation=0, fontsize=8):
    for p in ax.patches:
        h = p.get_height()
        x = p.get_x() + p.get_width()/2
        ax.text(x, h, fmt.format(h), ha="center", va="bottom", fontsize=fontsize, rotation=rotation)

def annotate_points(ax, x, y, step=7, fmt="{}", fontsize=7):
    """Подписываем каждую N-ю точку и обязательно последнюю.
    Индексируем ПО ПОЗИЦИИ (iloc), чтобы не ловить KeyError."""
    x = pd.Series(x).reset_index(drop=True)
    y = pd.Series(y).reset_index(drop=True)
    n = len(x)
    step = max(1, int(step))
    for i in range(0, n, step):
        ax.text(x.iloc[i], y.iloc[i], fmt.format(y.iloc[i]),
                fontsize=fontsize, ha="center", va="bottom")
    if n > 0:
        ax.text(x.iloc[-1], y.iloc[-1], fmt.format(y.iloc[-1]),
                fontsize=fontsize, ha="left", va="bottom")

# общие агрегаты (пригодятся для подписей)
total_visits = conv_daily["visits"].sum()
total_regs = conv_daily["registrations"].sum()

In [ ]:
plt.figure()
ax = plt.gca()
bars = ax.bar(["total"], [total_visits])
ax.set_title("Итоговые визиты (за период)")
ax.set_ylabel("Визитов")
annotate_bars(ax, fmt="{:,.0f}")
plt.tight_layout()
plt.savefig(CHARTS / "visits_total.png", dpi=150)
plt.show()

In [ ]:
by_plat_vis = merged.groupby("platform", as_index=False)["visits"].sum()

plt.figure()
ax = plt.gca()
ax.bar(by_plat_vis["platform"], by_plat_vis["visits"])
ax.set_title("Итоговые визиты по платформам")
ax.set_xlabel("Платформа")
ax.set_ylabel("Визитов")
annotate_bars(ax, fmt="{:,.0f}")
plt.tight_layout()
plt.savefig(CHARTS / "visits_by_platform_total.png", dpi=150)
plt.show()

In [ ]:
plt.figure()
ax = plt.gca()
ax.bar(["total"], [total_regs])
ax.set_title("Итоговые регистрации (за период)")
ax.set_ylabel("Регистраций")
annotate_bars(ax, fmt="{:,.0f}")
plt.tight_layout()
plt.savefig(CHARTS / "registrations_total.png", dpi=150)
plt.show()


In [ ]:
by_plat_regs = merged.groupby("platform", as_index=False)["registrations"].sum()

plt.figure()
ax = plt.gca()
ax.bar(by_plat_regs["platform"], by_plat_regs["registrations"])
ax.set_title("Итоговые регистрации по платформам")
ax.set_xlabel("Платформа")
ax.set_ylabel("Регистраций")
annotate_bars(ax, fmt="{:,.0f}")
plt.tight_layout()
plt.savefig(CHARTS / "registrations_by_platform_total.png", dpi=150)
plt.show()


In [ ]:
conv_pf = (
    merged.groupby(["date_group","platform"], as_index=False)[["visits","registrations"]].sum()
)
conv_pf["conversion"] = (conv_pf["registrations"] / conv_pf["visits"] * 100).replace([float("inf")], 0)

plt.figure()
ax = plt.gca()
for plat, sub in conv_pf.groupby("platform"):
    ax.plot(sub["date_group"], sub["conversion"], label=plat)

ax.set_title("Конверсия по платформам (по дням), %")
ax.set_xlabel("Дата")
ax.set_ylabel("Конверсия, %")
plt.xticks(rotation=45)
ax.legend(title="Платформа")
plt.tight_layout()
plt.savefig(CHARTS / "conversion_by_platform_daily.png", dpi=150)
plt.show()

In [ ]:
overall_conv = total_regs / total_visits * 100 if total_visits else 0

plt.figure()
ax = plt.gca()
ax.bar(["average"], [overall_conv])
ax.set_title("Средняя конверсия за период, %")
ax.set_ylabel("Конверсия, %")
annotate_bars(ax, fmt="{:.2f}%")
plt.tight_layout()
plt.savefig(CHARTS / "conversion_average.png", dpi=150)
plt.show()


In [ ]:
cost_daily = final.groupby("date_group", as_index=False)["cost"].sum()

plt.figure()
ax = plt.gca()
ax.plot(cost_daily["date_group"], cost_daily["cost"])
annotate_points(ax, cost_daily["date_group"], cost_daily["cost"], step=7, fmt="{:,.0f}")
ax.set_title("Затраты на рекламу по дням")
ax.set_xlabel("Дата")
ax.set_ylabel("Затраты")
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(CHARTS / "ads_cost_daily.png", dpi=150)
plt.show()


In [ ]:
plt.figure()
ax = plt.gca()
for camp, sub in final.groupby("utm_campaign"):
    ax.scatter(sub["date_group"], sub["visits"], label=camp, s=15)
    annotate_points(ax, sub["date_group"], sub["visits"], step=7, fmt="{:,.0f}", fontsize=7)

ax.set_title("Визиты с выделением рекламных кампаний")
ax.set_xlabel("Дата")
ax.set_ylabel("Визитов")
plt.xticks(rotation=45)
ax.legend(title="utm_campaign", fontsize=8)
plt.tight_layout()
plt.savefig(CHARTS / "visits_with_campaigns.png", dpi=150)
plt.show()

In [ ]:
plt.figure()
ax = plt.gca()
for camp, sub in final.groupby("utm_campaign"):
    ax.scatter(sub["date_group"], sub["registrations"], label=camp, s=15)
    annotate_points(ax, sub["date_group"], sub["registrations"], step=7, fmt="{:,.0f}", fontsize=7)

ax.set_title("Регистрации с выделением рекламных кампаний")
ax.set_xlabel("Дата")
ax.set_ylabel("Регистраций")
plt.xticks(rotation=45)
ax.legend(title="utm_campaign", fontsize=8)
plt.tight_layout()
plt.savefig(CHARTS / "registrations_with_campaigns.png", dpi=150)
plt.show()

In [14]:
import os, json, pandas as pd

print("Есть ли charts папка?:", os.path.isdir("./charts"))
print("Есть ли conversion.json?:", os.path.isfile("./conversion.json"))
print("Есть ли ads.json?:", os.path.isfile("./ads.json"))

# проверим формат и наличие дробной точности
j = json.loads(open("./conversion.json", "r", encoding="utf-8").read())
# покажем первые пару значений для контроля точности
list(j["conversion"].items())[:3]


Есть ли charts папка?: True
Есть ли conversion.json?: True
Есть ли ads.json?: True


[('0', 81.3333333333), ('1', 81.8181818182), ('2', 2.8673835125)]

In [17]:
import json, os, pandas as pd
print(os.path.isfile("./conversion.json"))
j = json.loads(open("./conversion.json","r",encoding="utf-8").read())
list(j["conversion"].items())[:3]  # д

True


[('0', 81.3333333333), ('1', 81.8181818182), ('2', 2.8673835125)]